In [ ]:
?print        = 打开print文档
Shift + Enter = 运行
Esc   + B     = 新建行
Esc   + M     = .md行
D     + D     = 删除本行

# contextlib
上下文管理

#### 读写文件结束后需要关闭文件

In [2]:
try:
    f = open('../files/try.txt', 'r')
    print(f.read())
finally:
    if f:
        f.close()

begin
name = try.txt
end



#### with语句会自动关闭对象

In [3]:
with open('../files/try.txt', 'r') as f:
    print(f.read())

begin
name = try.txt
end



#### 任何实现了上下文管理方法(\__enter__(), \__exit__())的对象都可以在with语句中自动关闭
```Python
with body as thing:
    do-something
```
1、执行 body 语句、生成上下文管理器  
2、调用上下文管理器的 __enter__ 方法、并将 __enter__ 方法的返回值赋值给 thing  
3、执行 do-something  
4、执行上下文管理器的 __exit__ 方法  
5、如果执行 do-something 出现正常、或 do-something 里面有 break/continue/return，调用 __exit__(None, None, None)  
6、如果执行 do-something 出现异常，调用 __exit__(exc_type, exc_value, traceback)，其中三个参数来自 sys.exc_info  
7、最后、 __exit__ 返回 False，则重新抛出异常、让 with 语句之外的语句处理；否则忽略异常  

In [17]:
class Query(object):
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        print('Begin')
        return self
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print('Error')
        else:
            print('End')
        return self
    def query(self):
        print('Query info about %s' % self.name)

In [18]:
with Query('Chen') as q:
    q.query()

Begin
Query info about Chen
End


#### contextlib提供了 @contextmanager 来外挂 \__enter__ 和 \__exit__ 功能
他把一个对象变成了上下文对象

In [22]:
from contextlib import contextmanager

class Query(object):
    def __init__(self, name):
        self.name = name
    def query(self):
        print('Query info about %s' % self.name)

@contextmanager
def create_query(name):
    print("Begin")
    q = Query(name)
    yield q
    print("End")
    # 看起来这个 装饰器 就是把自己执行两次
    # 可以用来在代码/对象前后自动执行特定代码

In [23]:
with create_query('Chen') as q:
    q.query()

Begin
Query info about Chen
End


#### 使用 @contextmanager 在对象/代码前后执行特定的代码

In [25]:
@contextmanager
def tag(text):
    print('<%s>' % text)
    yield
    print('<%s>' % text)

with tag('!!!'):
    print('hello')
    print('world')

# 先执行 yield 之前的语句
# 遇到 yield 时返回 with 语句并执行 with 语句(两个 print 语句)
# 最后执行 yield 之后的语句

<!!!>
hello
world
<!!!>


#### 使用 closing 将一个对象变成上下文对象

In [31]:
from contextlib     import closing
from urllib.request import urlopen

with closing(urlopen('https://www.python.org')) as python_page:
    count = 0
    for lines in python_page:
        print(lines)
        count += 1
        if count == 4:
            break

b'<!doctype html>\n'
b'<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n'
b'<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n'
b'<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n'


In [33]:
# 模仿一个 closing
@ contextmanager
def iclosing(do):
    try:
        yield do
    finally:
        do.close()